In [53]:
%run 01_hash.ipynb

In [54]:
import json
from copy import deepcopy

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [03_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/04_mine.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

It's `hash` is a unique fingerprint. Every time something in the tx changes, it gets recalculated.

In [55]:
class TX(Hashable): 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = float(value)
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.ctime()
        self.signed      = False
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop == 'sig': self.signed = True
    
    def smry(self): return f'{pmh(self.fr)} -> {pmh(self.to)} {self.value} eth'

Create random txs.

In [56]:
tx1 = TX(rh(), rh(), 12, 0.2,  0)
tx2 = TX(rh(), rh(), 6,  0.15, 0); print(tx1)

fr:            🕋 0xe50f1781ab...032
hash:          📨 0x828a0f3aa7...646
to:            📥 0x7fbe0521d3...7a4
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Tue Apr  6 00:00:43 2021
signed:        False


#### TX Summary

In [57]:
tx1.smry()

'🕋 0xe50f -> 📥 0x7fbe 12.0 eth'

### TX JSON

Every tx has a json representation that we will use for our API later.

In [58]:
tx1_json = tx1.json(); print(tx1_json)

{
    "fr": "0xe50f1781abbfc3fc2e02d354132bb9047374efffdb601065d8e92f51c1f06032",
    "hash": "0x828a0f3aa76f2d9f260d3fac77d480d65195695dc9485cd5843ea13125023646",
    "to": "0x7fbe0521d304e411a62477ad432963990c161405fd09e858c3549787580817a4",
    "value": 12.0,
    "fee": 0.2,
    "nonce": 0,
    "time": "Tue Apr  6 00:00:43 2021",
    "signed": false
}


We can load this JSON string `d` and create a tx object.

In [59]:
def load_tx(d):
    d = json.loads(d)
    tx = TX(d['fr'],d['to'],d['value'],d['fee'],d['nonce'])
    for k,v in d.items(): setattr(tx, k, v)
    tx.hash = d['hash']
    return tx

In [60]:
tx1_from_json = load_tx(tx1_json)
assert tx1 == tx1_from_json

#### Print several txs in a nice way.

In [61]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
print(txs2str([tx1,tx2]))

fr:            🕋 0xe50f1781ab...032
hash:          📨 0x828a0f3aa7...646
to:            📥 0x7fbe0521d3...7a4
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Tue Apr  6 00:00:43 2021
signed:        False

fr:            🔭 0xc7dffefac5...c03
hash:          🕢 0xfc5ae59f52...82f
to:            📷 0x917456e97d...a44
value:         6.0 eth
fee:           0.15 eth
nonce:         0
time:          Tue Apr  6 00:00:43 2021
signed:        False



#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [62]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2_false_value)

fr:            🔭 0xc7dffefac5...c03
hash:          📥 0x7fc7ba206c...06a
to:            📷 0x917456e97d...a44
value:         120
fee:           0.15 eth
nonce:         0
time:          Tue Apr  6 00:00:43 2021
signed:        False


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [63]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [64]:
tx1_signed = deepcopy(tx1)
tx1.sig    = rh(); print(tx1)

fr:            🕋 0xe50f1781ab...032
hash:          📨 0x828a0f3aa7...646
to:            📥 0x7fbe0521d3...7a4
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Tue Apr  6 00:00:43 2021
signed:        True
sig:           🔿 0xd947a5af0c...173


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [65]:
assert tx1 == tx1_signed